# Init Iceberg data
This notebook is mostly inspired by [Tabular.io blog post](https://tabular.medium.com/pyiceberg-0-2-1-pyarrow-and-duckdb-79effbd1077f) about PyIceberg. By running this notebook you will init the Iceberg data you will be able to query by Duckberg afterwards.

### Create database nyc

In [1]:
result = spark.sql("CREATE DATABASE IF NOT EXISTS nyc;")

### Create database nyc
Create Iceberg table that is partitioned by `payment_type`

In [2]:
query = """
CREATE TABLE IF NOT EXISTS nyc.taxis (
    VendorID              bigint,
    tpep_pickup_datetime  timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count       double,
    trip_distance         double,
    RatecodeID            double,
    store_and_fwd_flag    string,
    PULocationID          bigint,
    DOLocationID          bigint,
    payment_type          bigint,
    fare_amount           double,
    extra                 double,
    mta_tax               double,
    tip_amount            double,
    tolls_amount          double,
    improvement_surcharge double,
    total_amount          double,
    congestion_surcharge  double,
    airport_fee           double
)
USING iceberg
PARTITIONED BY (payment_type)
"""

result = spark.sql(query)

### Load data into table

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

for filename in [
    "yellow_tripdata_2022-01.parquet",
    "yellow_tripdata_2022-02.parquet",
    "yellow_tripdata_2022-03.parquet",
]:
    df = spark.read.parquet(f"/home/iceberg/data/{filename}")
    df.write.mode("append").saveAsTable("nyc.taxis")

23/12/20 09:54:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
